## takes ~2 days (CPU only) to fine-tune mT5 with 7000 sanskrit sentences

In [31]:
!pip install -q transformers datasets sentencepiece
# sentencepiece installation will require restarting kernel after installation to take effect

In [32]:
import sentencepiece
print(sentencepiece.__version__)


0.2.0


In [2]:
from transformers import (
    MT5ForConditionalGeneration,
    MT5Tokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
from datasets import load_dataset, Dataset
import numpy as np
import torch
import re
from transformers import AutoTokenizer

# Load mT5 Model and Tokenizer
# ===============================
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
MAX_LEN = 512

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [7]:
# ===============================
# Load and Clean Sanskrit Dataset
# ===============================

def clean_sanskrit_text(example):
    text = example["text"]
    
    # Remove zero-width characters and extra spaces
    text = re.sub(r'[\u200b-\u200d]', '', text)         # Remove zero-width characters
    text = re.sub(r'\s+', ' ', text)                    # Collapse multiple spaces/newlines
    text = text.strip()                                 # Trim leading/trailing whitespace
     # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    # Remove non-Sanskrit characters (retain Devanagari script)
    text = re.sub(r'[^\u0900-\u097F\s]', '', text)
   
    # Optional: Remove grammar tables (if undesired)
    grammar_markers = ["लट् लकार", "लङ् लकार", "प्रथमपुरुष", "मध्यमपुरुष", "उत्तमपुरुष"]
    for marker in grammar_markers:
        if marker in text:
            text = text.split(marker)[0].strip()
            break
    
    return {"text": text}

# Load and preprocess the dataset
dataset = load_dataset('oscar', 'unshuffled_deduplicated_sa', split='train[:100%]')

dataset = dataset.map(clean_sanskrit_text)
print(dataset[0])

Map:   0%|          | 0/7121 [00:00<?, ? examples/s]

{'id': 0, 'text': 'अनिरुद्धनगरे क्रीडिता रामलीला सम्प्रति समाप्ता अस्ति । तस्य कानिचन् चित्राणि पूर्वमेव प्रकाशितानि सन्ति । द्वौ चलचित्रौ अपि प्रकाशितौ । तस्मिन् एव क्रमेण एतत् सीतास्वयंबर इति चलचित्रं प्रकाश्यते ।'}


In [9]:
len(dataset)

7121

In [10]:
dataset[1]

{'id': 1,
 'text': 'पाठः क्रियेटिव कॉमन्स ऐट्रिब्यूशनशेयरअलाइक अभिज्ञापत्रस्य अन्तर्गततया उपलब्धः अस्ति अन्याः संस्थित्यः अपि सन्ति । अधिकं ज्ञातुम् अत्र उपयोगस्य संस्थितिं पश्यतु ।'}

In [11]:
dataset

Dataset({
    features: ['id', 'text'],
    num_rows: 7121
})

In [12]:
dataset[46]

{'id': 46,
 'text': 'क्रोधात् भवति सम्मोहः सम्मोहात् स्मृति विभ्रमः स्मृतिभृन्षात् बुद्धिनाशो बुद्धिनाशात् प्रनश्यते '}

In [13]:
# 5. Tokenize the Dataset
# ===============================
max_input_length = MAX_LEN
max_target_length = MAX_LEN

def preprocess(example):
    input_text = example["text"]
    input_ids = tokenizer(
        input_text,
        padding="max_length",
        truncation=True,
        max_length=max_input_length
    )

    labels = tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=max_target_length
    )

    input_ids["labels"] = labels["input_ids"]
    return input_ids

tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=["text"])


Map:   0%|          | 0/7121 [00:00<?, ? examples/s]

In [14]:
def preprocess_completion(examples):
    input_texts = ["complete: " + text[:100] for text in examples["text"]]
    target_texts = [text[100:200] if len(text) > 200 else text[-50:] for text in examples["text"]]  # dummy completion
    model_inputs = tokenizer(input_texts, padding="max_length", truncation=True, max_length=max_input_length)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(target_texts, padding="max_length", truncation=True, max_length=max_target_length)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [17]:
# Shared training arguments
def get_training_args(output_dir):
    return Seq2SeqTrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=2,
        num_train_epochs=2,
        learning_rate=5e-4,
        logging_dir="./logs",
        logging_steps=5000,
        save_steps=5000,
        save_total_limit=1,
        predict_with_generate=True,
        fp16=False
    )

# Trainer setup
def train_model(preprocess_fn, output_dir, remove_cols):
    tokenized_dataset = dataset.map(preprocess_fn, batched=True, remove_columns=remove_cols)
    training_args = get_training_args(output_dir)
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        tokenizer=tokenizer,
        data_collator=DataCollatorForSeq2Seq(tokenizer, model=model)
    )
    trainer.train()

# Task-specific training functions
def train_completion():
    print("Training for text completion...")
    train_model(preprocess_completion, "./mt5-sanskrit-completion", remove_cols=["text"])

# Evaluation functions
def evaluate(task, test_input):
    input_ids = tokenizer(test_input, return_tensors="pt").input_ids
    outputs = model.generate(input_ids=input_ids, max_length=512)
    print(f"\n{task} Result:\n", tokenizer.decode(outputs[0], skip_special_tokens=True))


In [ ]:
train_completion()

Training for text completion...


Map:   0%|          | 0/7121 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/var/tmp/ipykernel_19976/1713360364.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `pa

Step,Training Loss
5000,0.610600


In [19]:
 from transformers import (
    MT5ForConditionalGeneration,
    MT5Tokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
from datasets import load_dataset, Dataset
import numpy as np
import torch
import re
from transformers import AutoTokenizer

checkpoint_dir = "mt5-sanskrit-completion/checkpoint-5000/"  # your saved checkpoint directory

tokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)
model = MT5ForConditionalGeneration.from_pretrained(checkpoint_dir)


# for sentences in training data, has no problem completing them

In [22]:
evaluate("Text Completion", "अनिरुद्धनगरे क्रीडिता रामलीला सम्‍प्रति समाप्‍ता अस्ति ।")


Text Completion Result:
 । क्रीडिता रामलीला सम्प्रति समाप्ता अस्ति । क्रीडिता रामलीला सम्प्रति समाप्ता अस्ति । क्रीडिता रामलीला सम्प्रति समाप्ता अस्ति । क्रीडिता रामलीला सम्प्रति समाप्ता अस्ति । क्रीडिता रामलीला सम्प्रति समाप्ता अस्ति । क्रीडिता रामलीला सम्प्रति समाप्ता अस्ति । क्रीडिता रामलीला सम्प्रति समाप्ता अस्ति । क्रीडिता रामलीला सम्प्रति समाप्ता अस्ति । क्रीडिता रामलीला सम्प्रति समाप्ता अस्ति । क्रीडिता रामलीला सम्प्रति समाप्ता अस्ति । क्रीडिता रामलीला सम्प्रति समाप्ता अस्ति । क्रीडिता रामलीला सम्प्रति समाप्ता अस्ति । क्रीडिता रामलीला सम्प्रति समाप्ता अस्ति ।


# but fine-tuning for 1 day (and 7000 sanskrit sentences) insufficient to complete sentences for unseen data

In [20]:
evaluate("Text Completion", "मनुष्यस्य धर्म एव मुख्यः")


Text Completion Result:
 यस्य धर्म एव मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मुख्यः मु

In [21]:
evaluate("Text Completion", "पाठः क्रियेटिव कॉमन्स ऐट्रिब्यूशन/शेयर-अलाइक अभिज्ञापत्रस्य अन्तर्गततया उपलब्धः अस्ति")


Text Completion Result:
 यः अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्ति । अस्

In [23]:
evaluate("Text Completion", "सर्वे भवन्तु सुखिनः, सर्वे सन्तु निरामयाः।")


Text Completion Result:
 ् सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः सर्वे सन्तु निरामयाः। सर्वे सन्तु निरामयाः सर्वे सन्तु निरामयाः सर्वे सन्तु निरामयाः सर्वे सन्तु निरामयाः सर्वे सन्तु निरामयाः सर्वे सन्तु निरामयाः सर्वे सन्तु निरामयाः सर्वे सन्तु न

In [24]:
evaluate("Text Completion", "कर्मण्येवाधिकारस्ते मा फलेषु कदाचन मा कर्मफलहेतुर्भूर्मा ते सङ्गोऽस्त्वकर्मणि।")


Text Completion Result:
 ् । सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। सङ्गोऽस्त्वकर्मणि। स

In [25]:
!rm -r logs 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
